In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [2]:
# cd drive/MyDrive/IE7500_GroupB/Notebooks

In [6]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 21.4 MB/s eta 0:00:0000:0100:01


In [7]:
# load necesary libraries
import numpy as np
import pandas as pd

In [8]:
dtypes_dict = {'headline': 'object',
               'url': 'object',
               'publisher': 'object',
               'stock': 'object',
               'tokens': 'object',
               'normalized_tokens': 'object',
               'filtered_tokens': 'object',
               'lemmas': 'object',
               'sentiment_score': 'float64',
               'Name': 'object',
               'Market Cap': 'float64',
               'Country': 'object',
               'IPO Year': 'float64',
               'Sector': 'object',
               'Industry': 'object',
               'year': 'int32',
               'month': 'int32',
               'day_of_week': 'int32',
               'sentiment_label': 'int64',
               'headline_length': 'int64',
               'word_count': 'int64',
               'Market_Cap_Category': 'object',
               'recommendation': 'object',
               'cap_Large': 'bool',
               'cap_Medium': 'bool',
               'cap_Mega': 'bool',
               'cap_Micro': 'bool',
               'cap_Nano': 'bool',
               'cap_Small': 'bool',
               'sector_Basic Materials': 'bool',
               'sector_Consumer Discretionary': 'bool',
               'sector_Consumer Staples': 'bool',
               'sector_Energy': 'bool',
               'sector_Finance': 'bool',
               'sector_Health Care': 'bool',
               'sector_Industrials': 'bool',
               'sector_Miscellaneous': 'bool',
               'sector_Real Estate': 'bool',
               'sector_Technology': 'bool',
               'sector_Telecommunications': 'bool',
               'sector_Utilities': 'bool', 'recommendation_label': 'int64',
               'publisher_label': 'int64', 'country_label': 'int64',
               'industry_label': 'int64'}

In [9]:
# load dataframes to use
df_main = pd.read_csv("saved_dfs/df_for_models.csv", dtype=dtypes_dict,
                      parse_dates=['date'])

In [10]:
df_main.head()

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
0,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01,A,"['Agilent', 'Technologies', 'Announces', 'Pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technology', 'announces', 'pricin...",0.0000,...,True,False,False,False,False,False,1,4,45,18
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18,A,"['Agilent', '(', 'A', ')', 'Gears', 'Up', 'for...","['agilent', 'a', 'gears', 'up', 'for', 'q2', '...","['agilent', 'gears', 'q2', 'earnings', 'cards']","['agilent', 'gear', 'q2', 'earnings', 'card']",0.0000,...,True,False,False,False,False,False,1,16,45,18
2,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15,A,"['J.P.', 'Morgan', 'Asset', 'Management', 'Ann...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...",0.3612,...,True,False,False,False,False,False,1,4,45,18
3,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15,A,"['Pershing', 'Square', 'Capital', 'Management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...",0.0000,...,True,False,False,False,False,False,1,4,45,18
4,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12,A,"['Agilent', 'Awards', 'Trilogy', 'Sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'award', 'trilogy', 'science', 'go...",0.4588,...,True,False,False,False,False,False,1,4,45,18


Create Vectors from Headlines (Vectorization Models)

To generate vectors for the headline column, you can use different NLP vectorization techniques, including:

TF-IDF (Term Frequency - Inverse Document Frequency)

Count Vectorizer (Bag of Words)

Word2Vec (Pre-trained Word Embeddings)

Doc2Vec (Sentence-Level Embeddings)

BERT Embeddings (Transformer-based Representation)

In [13]:
import time
import joblib

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Feature selection
text_features = ["lemmas"]
numerical_features = ["sentiment_score"]
categorical_features = ["publisher_label", "country_label", "industry_label"]
sector_features = [
    "sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
    "sector_Technology", "sector_Telecommunications", "sector_Utilities"
]
target = "recommendation_label"

# Train/test split
X = df_main[text_features + numerical_features + categorical_features + sector_features]
y = df_main[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
tfidf = TfidfVectorizer(stop_words='english', max_features=8000, min_df=5, max_df=0.9)
num_scaler = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ("text", tfidf, "lemmas"),
        ("num", num_scaler, numerical_features),
        ("cat", "passthrough", categorical_features),
        ("sector", "passthrough", sector_features)
    ],
    n_jobs=-1
)


# Goal 1: Predict Buy / Hold / Sell

## Method 1: Logistic Regression

In [14]:
print("\nRunning Logistic Regression...")
start = time.time()

pipeline_lr = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=300, solver='saga', C=1.0, n_jobs=-1, random_state=42))
])

cv_scores = cross_val_score(pipeline_lr, X_train, y_train, cv=3, scoring="f1_macro", n_jobs=-1)
print(f"Logistic Regression Mean F1-score: {cv_scores.mean():.4f}")

pipeline_lr.fit(X_train, y_train)
y_pred = pipeline_lr.predict(X_test)
print("Logistic Regression Performance:")
print(classification_report(y_test, y_pred, zero_division=0))
joblib.dump(pipeline_lr, "logreg_pipeline.pkl")
print(f"Time: {time.time() - start:.2f} seconds")


Running Logistic Regression...


/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Logistic Regression Mean F1-score: 0.9537


/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     37064
           1       0.99      0.99      0.99    317370
           2       0.94      0.93      0.94     14678

    accuracy                           0.99    369112
   macro avg       0.96      0.96      0.96    369112
weighted avg       0.99      0.99      0.99    369112

Time: 743.59 seconds


## Method 2: RandomForest

In [15]:
print("\nRunning Random Forest...")
start = time.time()

pipeline_rf = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42))
])

cv_scores = cross_val_score(pipeline_rf, X_train, y_train, cv=3, scoring="f1_macro", n_jobs=-1)
print(f"Random Forest Mean F1-score: {cv_scores.mean():.4f}")

pipeline_rf.fit(X_train, y_train)
y_pred = pipeline_rf.predict(X_test)
print("Random Forest Performance:")
print(classification_report(y_test, y_pred, zero_division=0))
joblib.dump(pipeline_rf, "rf_pipeline.pkl")
print(f"Time: {time.time() - start:.2f} seconds")


Running Random Forest...
Random Forest Mean F1-score: 0.9998
Random Forest Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37064
           1       1.00      1.00      1.00    317370
           2       1.00      1.00      1.00     14678

    accuracy                           1.00    369112
   macro avg       1.00      1.00      1.00    369112
weighted avg       1.00      1.00      1.00    369112

Time: 545.10 seconds


## Method 3: SVM (Support Vector Machine)

In [18]:
from sklearn.linear_model import SGDClassifier

print("\nRunning SGDClassifier (Linear SVM)...")
start = time.time()

pipeline_sgd = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", SGDClassifier(
        loss="hinge",           # Linear SVM
        max_iter=1000,
        tol=1e-3,
        random_state=42,
        n_jobs=-1
    ))
])

cv_scores = cross_val_score(pipeline_sgd, X_train, y_train, cv=3, scoring="f1_macro", n_jobs=-1)
print(f"SGDClassifier Mean F1-score: {cv_scores.mean():.4f}")

pipeline_sgd.fit(X_train, y_train)
y_pred = pipeline_sgd.predict(X_test)
print("SGDClassifier Performance:")
print(classification_report(y_test, y_pred, zero_division=0))

joblib.dump(pipeline_sgd, "sgd_pipeline.pkl")
print(f"Time: {time.time() - start:.2f} seconds")



Running SGDClassifier (Linear SVM)...
SGDClassifier Mean F1-score: 0.8926
SGDClassifier Performance:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     37064
           1       0.98      1.00      0.99    317370
           2       0.95      0.80      0.87     14678

    accuracy                           0.98    369112
   macro avg       0.97      0.91      0.94    369112
weighted avg       0.98      0.98      0.98    369112

Time: 130.12 seconds
